In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.special import softmax
from scipy.stats import mode
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df = pd.read_csv('Potato-Prolific-Dataset-main/dataset/offensiveness/raw_data.csv')
df = df[df['race'] == 'White'][['text', 'offensiveness']].copy()
df = df.dropna(subset=['text','offensiveness'])
df['offensiveness'] = df['offensiveness'].astype(int)
df['labels'] = (df['offensiveness'] > 1).astype(int)

from sklearn.model_selection import train_test_split
train_val_df, test_df = train_test_split(
    df, test_size=0.2, stratify=df['labels'], random_state=42
)
test_texts = test_df['text'].tolist()
test_labels = test_df['labels'].values


In [ ]:
ensemble_models = [
    # RoBERTa 5
    ('bert1', 'bert-base-uncased', 'models/bert_seed42.bin'),
    ('bert2', 'bert-base-uncased', 'models/bert_seed128.bin'),
    ('bert3', 'bert-base-uncased', 'models/bert_seed256.bin'),
    ('bert4', 'bert-base-uncased', 'models/bert_seed1024.bin'),
    ('bert5', 'bert-base-uncased', 'models/bert_seed2025.bin'),
]
NUM_LABELS = 2
MAX_LENGTH = 128
BATCH_SIZE = 16


In [ ]:
all_logits = []   
model_names = []  

for (model_id, pretrained, ckpt) in ensemble_models:
    print(f"Predicting with {model_id} ({pretrained}) ...")
    tokenizer = AutoTokenizer.from_pretrained(pretrained)
    model = AutoModelForSequenceClassification.from_pretrained(pretrained, num_labels=NUM_LABELS)
    checkpoint = torch.load(ckpt, map_location=device)
    if "model_state_dict" in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
    else:
        model.load_state_dict(checkpoint)
    model.eval()
    model.to(device)

    logits_list = []
    with torch.no_grad():
        for i in tqdm(range(0, len(test_texts), BATCH_SIZE)):
            batch_text = test_texts[i:i+BATCH_SIZE]
            encoded = tokenizer(
                batch_text,
                padding='max_length',
                truncation=True,
                max_length=MAX_LENGTH,
                return_tensors='pt'
            )
            for k in encoded:
                encoded[k] = encoded[k].to(device)
            with torch.cuda.amp.autocast(enabled=False):
                out = model(**encoded)
                batch_logits = out.logits.cpu().numpy()
            logits_list.append(batch_logits)
    model_preds = np.concatenate(logits_list, axis=0)
    all_logits.append(model_preds)
    model_names.append(model_id)
    del model; torch.cuda.empty_cache()
print("All models predicted.")


In [ ]:
for i, name in enumerate(model_names):
    pred = all_logits[i].argmax(axis=1)
    print(f"\n=== Single Model {name} ===\n")
    print(classification_report(test_labels, pred, digits=4))

roberta_idx = [i for i,name in enumerate(model_names) if name.startswith('roberta')]
if roberta_idx:
    probs_roberta = [softmax(all_logits[i], axis=1) for i in roberta_idx]
    roberta_avgp = np.mean(probs_roberta,axis=0)
    r_pred = np.argmax(roberta_avgp, axis=1)
    print("\n=== All RoBERTa (soft vote ensemble) ===\n")
    print(classification_report(test_labels, r_pred, digits=4))

probs_list = [softmax(x, axis=1) for x in all_logits]
avg_probs = np.mean(probs_list, axis=0)
soft_preds = np.argmax(avg_probs, axis=1)
print("\n=== ALL RoBERTa models Soft Voting ===\n")
print(classification_report(test_labels, soft_preds, digits=4))

hard_preds_list = [np.argmax(x, axis=1) for x in all_logits]
hard_mat = np.stack(hard_preds_list, axis=0)
hard_preds = mode(hard_mat, axis=0).mode.squeeze()
print("\n=== ALL RoBERTa models Hard Voting ===\n")
print(classification_report(test_labels, hard_preds, digits=4))


In [ ]:
def plot_conf_matrix(y_true, y_pred, class_names, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap="Blues",
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(title)
    plt.show()

plot_conf_matrix(test_labels, soft_preds, class_names=["Non-offensive", "Offensive"], title="RoBERTa Soft Voting Confusion")
plot_conf_matrix(test_labels, hard_preds, class_names=["Non-offensive", "Offensive"], title="RoBERTa Hard Voting Confusion")
